In [329]:
#Import needed libraries

import yfinance as yf
from ta.momentum import RSIIndicator
from ta.trend import MACD
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [330]:
#Using Nvidia's stock data

nvda = yf.download("NVDA", start="2020-01-01", end="2020-12-31")
nvda.to_csv("nvda_data.csv")

C:\Users\daksh\AppData\Local\Temp\ipykernel_40924\2385680315.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nvda = yf.download("NVDA", start="2020-01-01", end="2020-12-31")
[*********************100%***********************]  1 of 1 completed


In [331]:
#Implementing features  

rsi = RSIIndicator(close = nvda["Close"]["NVDA"], window=14, fillna=False)
nvda["rsi"] = rsi.rsi()
nvda["macd"] = MACD(nvda["Close"]["NVDA"]).macd_diff()
nvda["returns"] = nvda["Close"]["NVDA"].pct_change()
nvda["ma_10"] = nvda["Close"]["NVDA"].rolling(10).mean()

In [332]:
#Check if the next day's closing price went up 

nvda["Target"] = (nvda["Close"]["NVDA"].shift(-1) > nvda["Close"]["NVDA"]).astype(int)

In [333]:
#Extracting data needed for training the model 

features = nvda[["rsi","macd","returns","ma_10"]].dropna()
labels = nvda["Target"].loc[features.index]

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=False)

In [334]:
#Initialize model and predict

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

model = LogisticRegression(class_weight='balanced')
model.fit(x_train_scaled, y_train)
y_pred = model.predict(x_test_scaled)

#print(y_train.value_counts())
#print(y_test.value_counts())
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.94      0.61        18
           1       0.83      0.19      0.31        26

    accuracy                           0.50        44
   macro avg       0.64      0.57      0.46        44
weighted avg       0.68      0.50      0.43        44



In [335]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(x_train, y_train)
y_pred = rf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.56      0.56        18
           1       0.69      0.69      0.69        26

    accuracy                           0.64        44
   macro avg       0.62      0.62      0.62        44
weighted avg       0.64      0.64      0.64        44



In [336]:
xgb_model = XGBClassifier()
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.39      0.50        18
           1       0.68      0.88      0.77        26

    accuracy                           0.68        44
   macro avg       0.69      0.64      0.63        44
weighted avg       0.69      0.68      0.66        44

